## Estructura

Claves:
1. Conexión con modelos
2. Conexión con datos
3. Encadenamiento de procesos


https://python.langchain.com/docs/integrations/providers/

# Fundamentos

In [3]:
from transformers import AutoTokenizer, pipeline
import torch

# model = "tiiuae/falcon-40b-instruct" # Generacion de texto
# model = "stabilityai/stablelm-tuned-alpha-3b"
model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer= tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)

/Users/kevinguillencisneros/Downloads/Developing_applications_with_Langchain/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.12s/it]
Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps


In [4]:
type(pipeline)

transformers.pipelines.text_generation.TextGenerationPipeline

In [5]:
from langchain import HuggingFacePipeline

llm_falcon = HuggingFacePipeline(
    pipeline = pipeline,
    model_kwargs = {
        "temperature": 0,
        "max_length": 200,
        "do_sample": True,
        "top_k": 10,
        "num_return_sequences": 1,
        "eos_token_id": tokenizer.eos_token_id
    }
)

/var/folders/0z/s0pfsfdn61b_3c4kn01_hktc0000gn/T/ipykernel_5028/4016280788.py:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm_falcon = HuggingFacePipeline(


In [6]:
llm_falcon

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x16142d7f0>, model_kwargs={'temperature': 0, 'max_length': 200, 'do_sample': True, 'top_k': 10, 'num_return_sequences': 1, 'eos_token_id': 11})

In [7]:
llm_falcon("what is AI?")

/var/folders/0z/s0pfsfdn61b_3c4kn01_hktc0000gn/T/ipykernel_5028/1947423762.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_falcon("what is AI?")
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'what is AI?\nAI stands for Artificial Intelligence. It is a branch of computer science that focuses on creating intelligent machines'

# Models with openAI

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPEN_AI_KEY")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [9]:
from langchain_openai import ChatOpenAI

llm_gpt3_5 = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    n = 1, #numero de diferentes respuestas
    temperature = 0.3
)

In [10]:
llm_gpt3_5

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x162c9d160>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x162ca9940>, root_client=<openai.OpenAI object at 0x1614c6e90>, root_async_client=<openai.AsyncOpenAI object at 0x162c9d2b0>, temperature=0.3, model_kwargs={}, openai_api_key=SecretStr('**********'), n=1)

In [11]:
llm_gpt3_5.invoke('Como puedo lograr una clase mas interaciva para estudiantes virtuales?')

AIMessage(content='Para lograr una clase más interactiva para estudiantes virtuales, puedes implementar las siguientes estrategias:\n\n1. Utilizar herramientas de videoconferencia que permitan la participación activa de los estudiantes, como la función de chat, la posibilidad de levantar la mano virtualmente para hacer preguntas o participar en la discusión, y la opción de compartir pantalla para mostrar trabajos o presentaciones.\n\n2. Fomentar la participación de los estudiantes a través de preguntas abiertas, encuestas en línea, debates virtuales y actividades colaborativas en tiempo real.\n\n3. Utilizar plataformas educativas interactivas que permitan a los estudiantes realizar actividades prácticas, como cuestionarios, juegos educativos, simulaciones y ejercicios de práctica.\n\n4. Incorporar recursos multimedia, como videos, infografías, podcasts y presentaciones interactivas, para mantener el interés de los estudiantes y facilitar la comprensión de los conceptos.\n\n5. Promover 

In [12]:
llm_davinci = ChatOpenAI(
    model_name = "text-davinci-003",
    n = 2, #numero de diferentes respuestas
    temperature = 0.3
)

In [13]:
generacion = llm_gpt3_5.generate(
    [
        "Dime un consejo de vida para alguien de 30 años",
        "Recomiendame libros similares a Hyperion Cantos"
    ]
)

In [14]:
generacion.generations[0]

[ChatGeneration(text='Nunca es tarde para empezar de nuevo. A los 30 años, aún tienes mucho tiempo por delante para alcanzar tus metas y sueños. No te compares con los demás, sigue tu propio camino y trabaja duro para lograr lo que deseas. Aprovecha cada oportunidad que se presente y nunca dejes de aprender y crecer como persona. Recuerda que la vida es un viaje y lo importante es disfrutar el camino. ¡No te rindas y sigue adelante!', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='Nunca es tarde para empezar de nuevo. A los 30 años, aún tienes mucho tiempo por delante para alcanzar tus metas y sueños. No te compares con los demás, sigue tu propio camino y trabaja duro para lograr lo que deseas. Aprovecha cada oportunidad que se presente y nunca dejes de aprender y crecer como persona. Recuerda que la vida es un viaje y lo importante es disfrutar el camino. ¡No te rindas y sigue adelante!', additional_kwargs={'refusal': None}, response_metadata={

In [15]:
generacion.generations[1]

[ChatGeneration(text='1. "Dune" by Frank Herbert - A classic science fiction novel that explores themes of power, politics, and religion in a distant future.\n\n2. "The Expanse series" by James S.A. Corey - A series of science fiction novels that follow the crew of a spaceship as they navigate political intrigue and interstellar conflict.\n\n3. "Altered Carbon" by Richard K. Morgan - A cyberpunk novel that explores themes of identity, consciousness, and technology in a future where people can transfer their consciousness between bodies.\n\n4. "The Culture series" by Iain M. Banks - A series of science fiction novels that explore a post-scarcity society run by advanced artificial intelligences.\n\n5. "The Commonwealth Saga" by Peter F. Hamilton - A series of science fiction novels that follow humanity as they colonize the galaxy and encounter alien civilizations.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='1. "Dune" by Frank Herbert - A clas

In [16]:
generacion.llm_output

{'token_usage': {'completion_tokens': 286,
  'prompt_tokens': 40,
  'total_tokens': 326,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-3.5-turbo'}

In [17]:
llm_gpt3_5.get_num_tokens('Mis jefes se van a preocupar si gasto mucho en openai')

16

# Prompt templates

In [18]:
"""
Instrucciones:
Información externa o contexto:
Entrada del usuario o consulta:
Indicador de salida:
"""

'\nInstrucciones:\nInformación externa o contexto:\nEntrada del usuario o consulta:\nIndicador de salida:\n'

In [19]:
prompt_argentino = """ 

Respondé la pregunta basándote en el contexto de abajo. Si la
pregunta no puede ser respondida usando la información proporcionada,
respondé con "Ni idea, che".

Contexto: Los Modelos de Lenguaje de Gran Escala (MLGEs) son lo último en modelos usados en el Procesamiento del Lenguaje Natural (NLP).
Su desempeño superior a los modelos más chicos los hizo increíblemente
útiles para los desarrolladores que arman aplicaciones con NLP. Estos modelos
se pueden acceder vía la librería `transformers` de Hugging Face, vía OpenAI
usando la librería `openai`, y vía Cohere usando la librería `cohere`.

Pregunta: ¿Qué librerías están cerca de Buenos Aires?

Respuesta (escribe como argentina informal): 

"""

print(llm_gpt3_5(prompt_argentino))

/var/folders/0z/s0pfsfdn61b_3c4kn01_hktc0000gn/T/ipykernel_5028/1643221544.py:19: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm_gpt3_5(prompt_argentino))


content='Ni idea, che.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 198, 'total_tokens': 204, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BOxLRynhRB6tfO5xWWJRgOVKUzg0d', 'finish_reason': 'stop', 'logprobs': None} id='run-7cc059bf-4bde-4ba8-926b-4a317af80e9e-0' usage_metadata={'input_tokens': 198, 'output_tokens': 6, 'total_tokens': 204, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [20]:
from langchain import PromptTemplate

plantilla_colombiana = """
Responde a la pregunta con base en el siguiente contexto, parce. Si la
pregunta no puede ser respondida con la información proporcionada, responde
con "No tengo ni idea, ome".

Contexto: Los Modelos de Lenguaje de Gran Escala (MLGEs) son lo último en modelos usados en el Procesamiento del Lenguaje Natural (NLP).
Su desempeño superior a los modelos más chicos los hizo increíblemente
útiles para los desarrolladores que arman aplicaciones con NLP. Estos modelos
se pueden acceder vía la librería `transformers` de Hugging Face, vía OpenAI
usando la librería `openai`, y vía Cohere usando la librería `cohere`.

Pregunta: {query}

Respuesta (escribe como colombiano informal): 
"""

prompt_plantilla_colombiana = PromptTemplate(
    input_varibales = ["query"],
    template = plantilla_colombiana
)

In [21]:
from langchain import LLMChain

llm_gpt3_5_chain = LLMChain(
    prompt = prompt_plantilla_colombiana,
    llm = llm_gpt3_5
)

pregunta = "Que son os LLMs"

llm_gpt3_5_chain.run(pregunta)

/var/folders/0z/s0pfsfdn61b_3c4kn01_hktc0000gn/T/ipykernel_5028/4108755614.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_gpt3_5_chain = LLMChain(
/var/folders/0z/s0pfsfdn61b_3c4kn01_hktc0000gn/T/ipykernel_5028/4108755614.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_gpt3_5_chain.run(pregunta)


'Los LLMs son los Modelos de Lenguaje de Gran Escala, parce.'

In [23]:
pregunta = "What is Vallenato"

llm_gpt3_5_chain.run(pregunta)

'¡El Vallenato es un género musical típico de la costa caribeña de Colombia, parce! ¡Es una mezcla de acordeón, caja y guacharaca que te pone a bailar al instante! ¡Una delicia, ome!'

# Cadenas

In [ ]:
"""
Son el corazon de langchain

Secuencias de eslabones

Retorna una repsuesta que usa un LLM
"""